In [1]:
import os
import sys
import time
import pandas as pd
import numpy as np
os.environ['KERAS_BACKEND'] = "tensorflow"
import keras as K
import tensorflow
import multiprocessing
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.densenet import DenseNet121, preprocess_input
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, Callback, ModelCheckpoint
from keras.layers import Dense
from keras.models import Model
from keras.utils import multi_gpu_model
from sklearn.metrics.ranking import roc_auc_score
from sklearn.model_selection import train_test_split
from common.utils import *

Using TensorFlow backend.


In [2]:
# Performance Improvement
# 1. Make sure channels-first (not last)
K.backend.set_image_data_format('channels_first')

In [3]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Keras: ", K.__version__)
print("Numpy: ", np.__version__)
print("Tensorflow: ", tensorflow.__version__)
print(K.backend.backend())
print(K.backend.image_data_format())
print("GPU: ", get_gpu_name())
print(get_cuda_version())
print("CuDNN Version ", get_cudnn_version())

OS:  linux
Python:  3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
Keras:  2.1.4
Numpy:  1.14.1
Tensorflow:  1.4.0
tensorflow
channels_first
GPU:  ['Tesla P100-PCIE-16GB', 'Tesla P100-PCIE-16GB']
CUDA Version 8.0.61
CuDNN Version  6.0.21


In [4]:
# User-set
# Note if NUM_GPUS > 1 then MULTI_GPU = True and ALL GPUs will be used
# Set below to affect batch-size
# E.g. 1 GPU = 64, 2 GPUs = 64*2, 4 GPUs = 64*4
# Note that the effective learning-rate will be decreased this way
NUM_GPUS = 2 # Scaling factor for batch
NUM_CPUS = multiprocessing.cpu_count()
print(NUM_CPUS)

12


In [5]:
# Globals
CLASSES = 14
WIDTH = 224
HEIGHT = 224
CHANNELS = 3
LR = 0.0001  # Effective learning-rate will decrease as BATCHSIZE rises
EPOCHS = 5
BATCHSIZE = 64*NUM_GPUS
#IMAGENET_RGB_MEAN = [0.485, 0.456, 0.406]
#IMAGENET_RGB_SD = [0.229, 0.224, 0.225]
TOT_PATIENT_NUMBER = 30805  # From data

In [6]:
# Paths
CSV_DEST = "chestxray"
IMAGE_FOLDER = os.path.join(CSV_DEST, "images")
LABEL_FILE = os.path.join(CSV_DEST, "Data_Entry_2017.csv")
print(IMAGE_FOLDER, LABEL_FILE)

chestxray/images chestxray/Data_Entry_2017.csv


In [7]:
%%time
# Download data
print("Please make sure to download")
print("https://docs.microsoft.com/en-us/azure/storage/common/storage-use-azcopy-linux#download-and-install-azcopy")
download_data_chextxray(CSV_DEST)

Please make sure to download
https://docs.microsoft.com/en-us/azure/storage/common/storage-use-azcopy-linux#download-and-install-azcopy
Data already exists
CPU times: user 699 ms, sys: 220 ms, total: 919 ms
Wall time: 919 ms


In [8]:
#####################################################################################################
## Data Loading

In [9]:
class XrayData():
    
    def __init__(self, img_dir, lbl_file, patient_ids, 
                 width=WIDTH, height=HEIGHT, batch_size=BATCHSIZE, num_classes=CLASSES,
                 shuffle=True, seed=None, augment=False):
        
        self.patient_ids = patient_ids
        self.lbl_file = lbl_file
        
        # Hack for flow_from_directory to work, give it path above
        self.child_path  = os.path.split(img_dir)[-1]
        self.parent_path =  img_dir.replace(self.child_path,'')
        
        # Create ImageDataGenerator with DenseNet pre-processing
        # imagenet_utils.preprocess_input(x, data_format, mode='torch')
        if augment:
            datagen = ImageDataGenerator(
                horizontal_flip=True,
                # Best match to?
                # transforms.RandomResizedCrop(size=WIDTH),
                zoom_range=0.2,  
                rotation_range=10,
                preprocessing_function=preprocess_input)
        else:
             datagen = ImageDataGenerator(preprocessing_function=preprocess_input)    

        # Create flow-from-directory
        flowgen = datagen.flow_from_directory(
            directory=self.parent_path,  # hack: this is one directory up
            target_size=(width, height),
            batch_size=batch_size,
            shuffle=shuffle,
            seed=seed,
            class_mode='binary')    
        
        # Override previously created classes variables
        # filenames, classes
        flowgen.filenames, flowgen.classes = self._override_classes()
        # number of files
        flowgen.n = len(flowgen.filenames)
        # number of classes (not sure if this last one needed)
        flowgen.num_classes = num_classes
        
        self.generator = flowgen
        print("Loaded {} labels and {} images".format(len(self.generator.classes), 
                                                      len(self.generator.filenames)))

    
    def _override_classes(self):
        # Read labels-csv
        df = pd.read_csv(self.lbl_file)

        # Split labels on unfiltered data
        df_label = df['Finding Labels'].str.split(
            '|', expand=False).str.join(sep='*').str.get_dummies(sep='*')
        
        # Filter by patient-ids (both)
        df_label['Patient ID'] = df['Patient ID']
        
        df_label = df_label[df_label['Patient ID'].isin(self.patient_ids)]
        df = df[df['Patient ID'].isin(self.patient_ids)]

        # Remove unncessary columns
        df_label.drop(['Patient ID','No Finding'], axis=1, inplace=True)
        
        # List of images       
        img_locs =  df['Image Index'].map(lambda im: os.path.join(self.child_path, im)).values
        labels = df_label.values
        # Return new file-names and labels
        return img_locs, labels

In [10]:
# Training / Valid / Test split (70% / 10% / 20%)
train_set, other_set = train_test_split(
    range(1,TOT_PATIENT_NUMBER+1), train_size=0.7, test_size=0.3, shuffle=False)
valid_set, test_set = train_test_split(other_set, train_size=1/3, test_size=2/3, shuffle=False)
print("train:{} valid:{} test:{}".format(
    len(train_set), len(valid_set), len(test_set)))

train:21563 valid:3080 test:6162


In [11]:
train_dataset = XrayData(IMAGE_FOLDER, LABEL_FILE, train_set, augment=True).generator

Found 112120 images belonging to 1 classes.
Loaded 87306 labels and 87306 images


In [12]:
valid_dataset = XrayData(IMAGE_FOLDER, LABEL_FILE, valid_set, shuffle=False).generator
test_dataset = XrayData(IMAGE_FOLDER, LABEL_FILE, test_set, shuffle=False).generator

Found 112120 images belonging to 1 classes.
Loaded 7616 labels and 7616 images
Found 112120 images belonging to 1 classes.
Loaded 17198 labels and 17198 images


In [13]:
#####################################################################################################
## Helper Functions

In [14]:
def get_symbol(model_name='densenet121', out_features=CLASSES):
    # Recommended to instantiate base model on CPU
    # https://keras.io/utils/#multi_gpu_model
    # Yet another Keras hack ...
    with tensorflow.device('/cpu:0'):
        if model_name == 'densenet121':
            model = DenseNet121(input_shape=(3, 224, 224), weights='imagenet', include_top=False, pooling='avg')
        else:
            raise ValueError("Unknown model-name")
        # Add classifier to model FC-14
        classifier = Dense(out_features, activation='sigmoid')(model.output)
        model = Model(inputs=model.input, outputs=classifier)
    return model

In [15]:
def init_symbol(sym, lr=LR):
    # BCE Loss since classes not mutually exclusive + Sigmoid FC-layer
    sym.compile(
        loss = "binary_crossentropy",
        optimizer = Adam(lr, beta_1=0.9, beta_2=0.999, epsilon=None))
    # Callbacks
    sch = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1)
    #This doesnt work with Keras multi-gpu
    #Don't want to add another hack to get it fixed
    #chp = ModelCheckpoint('best_chexnet.pth.hdf5', monitor='val_loss', save_weights_only=False)
    callbacks = [sch]
    return sym, callbacks

In [16]:
def compute_roc_auc(data_gt, data_pd, full=True, classes=CLASSES):
    roc_auc = []
    for i in range(classes):
        roc_auc.append(roc_auc_score(data_gt[:, i], data_pd[:, i]))
    print("Full AUC", roc_auc)
    roc_auc = np.mean(roc_auc)
    return roc_auc

In [17]:
#####################################################################################################
## Train CheXNet

In [18]:
%%time
# Load symbol
chexnet_sym = get_symbol()

CPU times: user 20 s, sys: 1.47 s, total: 21.4 s
Wall time: 21 s


In [19]:
%%time
# Load optimiser, loss
multi_gpu_sym = multi_gpu_model(chexnet_sym, gpus=NUM_GPUS)
model, callbacks = init_symbol(multi_gpu_sym)

CPU times: user 25.3 s, sys: 460 ms, total: 25.8 s
Wall time: 25.4 s


In [20]:
%%time
# Training loop: 32m1s
model.fit_generator(train_dataset,
                    epochs=EPOCHS,
                    verbose=1,
                    callbacks=callbacks,
                    workers=NUM_CPUS,  # Num of CPUs since multiprocessing
                    use_multiprocessing=True,  # Faster than with threading
                    validation_data=valid_dataset,
                    max_queue_size=20)  # Default is 10 (most prob no difference)

Epoch 1/5
683/683 [==============================] - 409s 599ms/step - loss: 0.2042 - val_loss: 0.1626
Epoch 2/5
683/683 [==============================] - 372s 544ms/step - loss: 0.1512 - val_loss: 0.1516
Epoch 3/5
683/683 [==============================] - 372s 545ms/step - loss: 0.1470 - val_loss: 0.1440
Epoch 4/5
683/683 [==============================] - 372s 545ms/step - loss: 0.1439 - val_loss: 0.1395
Epoch 5/5
683/683 [==============================] - 372s 545ms/step - loss: 0.1413 - val_loss: 0.1392
CPU times: user 1h 3min 6s, sys: 16min 16s, total: 1h 19min 23s
Wall time: 32min 1s


In [21]:
#####################################################################################################
## Test CheXNet

In [22]:
# Load model for testing
# Currently multi-GPU checkpointing is broken on Keras
# For now use in-RAM model

In [23]:
%%time
## Evaluate
# AUC: 0.8174
y_guess = model.predict_generator(test_dataset, workers=NUM_CPUS)

CPU times: user 5min 15s, sys: 1min 15s, total: 6min 30s
Wall time: 1min 35s


In [24]:
print("Validation AUC: {0:.4f}".format(compute_roc_auc(test_dataset.classes, y_guess)))

Full AUC [0.8012330408091335, 0.8819616001902157, 0.8029702943084248, 0.9012762687220458, 0.8867431240008598, 0.9347619456118644, 0.7398953485383628, 0.8828213191420738, 0.6356629133353404, 0.8258513938077688, 0.7226508576695077, 0.7966816054853866, 0.7666252329668405, 0.8903557386233674]
Validation AUC: 0.8192


In [25]:
#####################################################################################################
## Extra: IO Experiment (time on numpy arrays)

In [42]:
def data_gen_to_numpy(data_gen, bs=BATCHSIZE, ch=CHANNELS, wi=WIDTH, hi=HEIGHT, cl=CLASSES):
    n_batches = ((data_gen.n-1)//bs)+1
    x_dta = np.zeros((n_batches*bs, ch, wi, hi), 
                   dtype=np.float32)
    y_dta = np.zeros((n_batches*bs, cl),
                   dtype=np.int32)
    c = 0
    for x, y in data_gen:
        ln = len(y)
        x_dta[c*ln:(c+1)*ln] = x
        y_dta[c*ln:(c+1)*ln] = y
        c+= 1
        if c == n_batches:
            return x_dta, y_dta

In [45]:
%%time
# Single-threaded -> ouch
x_train, y_train = data_gen_to_numpy(train_dataset)

CPU times: user 33min 21s, sys: 58.6 s, total: 34min 20s
Wall time: 33min 48s


In [43]:
%%time
x_val, y_val = data_gen_to_numpy(valid_dataset)

CPU times: user 1min 45s, sys: 4.7 s, total: 1min 50s
Wall time: 1min 48s


In [46]:
print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)

(87424, 3, 224, 224) (87424, 14)
(7680, 3, 224, 224) (7680, 14)


In [49]:
%%time
# Time for one epoch is 366s vs 372s with data-generator (6s of IO lag)
model.fit(x_train,
          y_train,
          epochs=1,
          batch_size=BATCHSIZE,
          verbose=1,
          callbacks=callbacks,
          validation_data=(x_val, y_val))

Train on 87424 samples, validate on 7680 samples
Epoch 1/1
87424/87424 [==============================] - 367s 4ms/step - loss: 0.1382 - val_loss: 0.1367
CPU times: user 12min 50s, sys: 2min 44s, total: 15min 35s
Wall time: 6min 6s
